## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

#### Exercise 1.1


In [57]:
from datasets import load_dataset, get_dataset_split_names

In [58]:
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

splits = get_dataset_split_names("cornell-movie-review-data/rotten_tomatoes")
print("Splits:", splits)

Splits: ['train', 'validation', 'test']


In [59]:
print("Train size:", len(dataset["train"]))
print("Validation size:", len(dataset["validation"]))
print("Test size:", len(dataset["test"]))

Train size: 8530
Validation size: 1066
Test size: 1066


In [60]:
print("Example:\n", dataset["train"][211])

Example:
 {'text': '[a] rare , beautiful film .', 'label': 1}


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [64]:
from transformers import AutoTokenizer, AutoModel

name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModel.from_pretrained(name)

sample = dataset["test"]["text"][260]
print(sample)

encoded_input = tokenizer(sample, return_tensors="pt")
print(encoded_input)

intensely romantic , thought-provoking and even an engaging mystery .
{'input_ids': tensor([[  101, 20531,  6298,  1010,  2245,  1011,  4013, 22776,  1998,  2130,
          2019, 11973,  6547,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [67]:
output = model(**encoded_input)
print(output)

BaseModelOutput(last_hidden_state=tensor([[[-0.2474, -0.2550, -0.1434,  ..., -0.2345,  0.4026,  0.1096],
         [-0.0094,  0.2953,  0.0051,  ..., -0.3797,  0.2477, -0.3513],
         [-0.2699,  0.1544,  0.3303,  ..., -0.5992,  0.1053, -0.4761],
         ...,
         [-0.1478, -0.0413,  0.4556,  ..., -0.2515,  0.2923, -0.3443],
         [-0.5207, -0.7747, -0.1066,  ...,  0.3429,  0.4019, -0.4375],
         [ 0.9744, -0.0602, -0.1639,  ..., -0.0061, -0.3364, -0.0374]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [68]:
print("\nLast hidden state shape:", output.last_hidden_state.shape)


Last hidden state shape: torch.Size([1, 15, 768])


In [69]:
sample = dataset["test"]["text"][29]
print(sample)

encoded_input = tokenizer(sample, return_tensors="pt")

output = model(**encoded_input)
print("\nLast hidden state shape:", output.last_hidden_state.shape)

a soul-stirring documentary about the israeli/palestinian conflict as revealed through the eyes of some children who remain curious about each other against all odds .

Last hidden state shape: torch.Size([1, 31, 768])


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [70]:
sample = "i want to see if [CLS] is at the beginning or at the end of the sequence"
encoded = tokenizer(sample, return_tensors="pt")

print("Token IDs:", encoded["input_ids"])
print("Decoded tokens:", tokenizer.convert_ids_to_tokens(encoded["input_ids"][0]))

Token IDs: tensor([[ 101, 1045, 2215, 2000, 2156, 2065,  101, 2003, 2012, 1996, 2927, 2030,
         2012, 1996, 2203, 1997, 1996, 5537,  102]])
Decoded tokens: ['[CLS]', 'i', 'want', 'to', 'see', 'if', '[CLS]', 'is', 'at', 'the', 'beginning', 'or', 'at', 'the', 'end', 'of', 'the', 'sequence', '[SEP]']


#### So the [CLS] is the first element of the output sequence!

In [76]:
from tqdm import tqdm
import torch

def extract_features(texts, tokenizer, model, batch_size=256, device="cuda"):
    model = model.to(device)
    model.eval()  

    num_texts = len(texts)
    hidden_size = 768
    features = torch.zeros((num_texts, hidden_size))

    with torch.no_grad():
        for start in tqdm(range(0, num_texts, batch_size), desc="Extracting features"):
            end = start + batch_size
            batch = texts[start:end]

            encoded = tokenizer(
                batch,
                padding=True,
                return_tensors="pt"
            ).to(device)

            output = model(**encoded)

            cls_vec = output.last_hidden_state[:, 0, :] 

            features[start:end, :] = cls_vec.cpu()

    return features

In [77]:
text_train = extract_features(dataset["train"]["text"], tokenizer, model)
labels_train = dataset["train"]["label"]

text_validation = extract_features(dataset["validation"]["text"], tokenizer, model)
labels_validation = dataset["validation"]["label"]

text_test = extract_features(dataset["test"]["text"], tokenizer, model)
labels_test = dataset["test"]["label"]


Extracting features: 100%|██████████| 5/5 [00:09<00:00,  1.82s/it]


In [78]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

clf = LinearSVC()
clf.fit(text_train, labels_train)

validation_predictions = clf.predict(text_validation)
test_predictions = clf.predict(text_test)

print(f"Validation accuracy: {accuracy_score(labels_validation, validation_predictions):.3f}")
print(f"Test accuracy: {accuracy_score(labels_test, test_predictions):.3f}")

Validation accuracy: 0.822
Test accuracy: 0.798


-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [82]:
def tokenize(dataset_element):
    encoded = tokenizer(dataset_element["text"])

    return {
        "text": dataset_element["text"],
        "label": dataset_element["label"],
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
    }

In [83]:
tokenized_train = dataset["train"].map(tokenize)
tokenized_val = dataset["validation"].map(tokenize)
tokenized_test = dataset["test"].map(tokenize)

In [88]:
print(tokenized_test[29])

{'text': 'a soul-stirring documentary about the israeli/palestinian conflict as revealed through the eyes of some children who remain curious about each other against all odds .', 'label': 1, 'input_ids': [101, 1037, 3969, 1011, 18385, 4516, 2055, 1996, 5611, 1013, 9302, 4736, 2004, 3936, 2083, 1996, 2159, 1997, 2070, 2336, 2040, 3961, 8025, 2055, 2169, 2060, 2114, 2035, 10238, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [1]:
# Your code here.

#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [2]:
# Your code here.

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [3]:
# Your code here.

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [4]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [5]:
# Your code here.